In [1]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK for Python: Custom training using Python package, managed text dataset, and TF Serving container
<table align="left">

  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fsdk%2FSDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td> 
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/sdk/SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      Open in Vertex AI Workbench
    </a>
  </td> 
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      View on GitHub
    </a>
  </td>                                                                                              
</table>


## Overview

This notebook demonstrates how to create a Custom Model using Custom Python Package Training, with a Vertex AI Dataset, and how to serve the model using TensorFlow-Serving Container for online prediction, and batch prediction. It requires you to provide a bucket to store the dataset.

Note: You may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training).

### Objective

In this tutorial, you learn how to create a Custom Model using Custom Python Package Training and you learn how to serve the model using TensorFlow-Serving Container for online prediction. Then you perform batch prediction on the model. 

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Dataset
- Veretx AI CustomPythonPackageTrainingJob
- Vertex AI model resource
- Vertex AI endpoint resource
- Vertex AI Batch Prediction

The steps performed include:

- Create utility functions to download data and prepare csv files for creating Vertex AI Managed    Dataset
- Download Data
- Prepare CSV Files for Creating Managed Dataset
- Create Custom Training Python Package
- Create TensorFlow Serving Container
- Run Custom Python Package Training with Managed Text Dataset
- Deploy a Model and Create an Endpoint on Vertex AI
- Predict on the Endpoint
- Create a Batch Prediction Job on the Model

### Dataset
#### Stack Overflow Data
You download the stack overflow data from from  https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz and create a Vertex AI managed text dataset. 

The Stack Overflow Data is licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License. To view a copy of this license, visit http://creativecommons.org/licenses/by-sa/3.0/ 

For more information about this dataset please visit: https://console.cloud.google.com/marketplace/details/stack-exchange/stack-overflow

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Get started

### Install Vertex AI SDK for Python and other required packages


In [15]:
! pip3 uninstall google-cloud-aiplatform -y

Found existing installation: google-cloud-aiplatform 1.58.0
Uninstalling google-cloud-aiplatform-1.58.0:
  Successfully uninstalled google-cloud-aiplatform-1.58.0


In [16]:
IS_WORKBENCH_NOTEBOOK

'/opt/conda'

In [19]:
! pip3 uninstall shapely -y

Found existing installation: shapely 2.0.4
Uninstalling shapely-2.0.4:
  Successfully uninstalled shapely-2.0.4


In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade shapely google-cloud-aiplatform tensorflow==2.15 {USER_FLAG} -q


### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [3]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [4]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**



When you submit a training job using the Vertex AI SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. Vertex AI runs
the code from this package. In this tutorial, Vertex AI also saves the
trained model that results from your job in the same bucket. Using this model artifact, you can then
create Vertex AI model and endpoint resources in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [3]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [7]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

Creating gs://k-bucket-vertexai-service-project-sdk-tf-managed/...


### Initialize Vertex AI SDK for Python

In [7]:
! pip install --upgrade google-cloud-aiplatform[full]==1.56.0

In [8]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION,staging_bucket=BUCKET_URI)

ModuleNotFoundError: No module named 'google.cloud.aiplatform_v1beta1.services.dataset_service'

### Import libraries and define constants

In [9]:
import csv
import os

from google.cloud import storage
from tensorflow.keras import utils

2024-07-03 11:14:25.836429: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-03 11:14:25.848683: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-03 11:14:25.887081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 11:14:25.964120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 11:14:25.964185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-03 11:14:26.013383: I tensorflow/core/platform/cpu_feature_guard.cc:

### Set Your Application Name, Task Name, and Directories.


In [109]:
APP_NAME = "keras-text-class-stack-overflow-tag"
TASK_TYPE = "mbsdk_custom-py-pkg-training"

TASK_NAME = f"{TASK_TYPE}_{APP_NAME}"

TASK_DIR = f"./{TASK_NAME}"
DATA_DIR = f"{TASK_DIR}/data"

print(f"Task Name:      {TASK_NAME}")
print(f"Task Directory: {TASK_DIR}")
print(f"Data Directory: {DATA_DIR}")

Task Name:      mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag
Task Directory: ./mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag
Data Directory: ./mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag/data


### Set a GCS prefix

If you want to centeralize all input and output files under the gcs location.

In [11]:
BUCKET_NAME = BUCKET_URI.split("gs://")[1]
GCS_PREFIX = f"{TASK_TYPE}/{APP_NAME}"

print(f"Bucket Name:    {BUCKET_NAME}")
print(f"GCS Prefix:     {GCS_PREFIX}")

Bucket Name:    k-bucket-vertexai-service-project-sdk-tf-managed
GCS Prefix:     mbsdk_custom-py-pkg-training/keras-text-class-stack-overflow-tag


### Utility functions to download data and prepare csv files for creating Vertex AI managed dataset

In [107]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    destination_file_name = os.path.join("gs://", bucket_name, destination_blob_name)

    return destination_file_name


def download_data(data_dir):
    """Download data."""

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
    dataset = utils.get_file(
        "stack_overflow_16k.tar.gz",
        url,
        untar=True,
        cache_dir=data_dir,
        cache_subdir="",
    )
    data_dir = os.path.join(os.path.dirname(dataset))

    return data_dir


def upload_train_data_to_gcs(train_data_dir, bucket_name, destination_blob_prefix):
    """Create CSV file using train data content."""

    train_data_dir = os.path.join(data_dir, "train")
    train_data_fn = os.path.join(data_dir, "train.csv")

    fp = open(train_data_fn, "w", encoding="utf8")
    writer = csv.writer(
        fp, delimiter=",", quotechar='"', quoting=csv.QUOTE_ALL, lineterminator="\n"
    )

    for root, _, files in os.walk(train_data_dir):
        for file in files:
            if file.endswith(".txt"):
                class_name = root.split("/")[-1]
                file_fn = os.path.join(root, file)
                with open(file_fn) as f:
                    content = f.readlines()
                    lines = [x.strip().strip('"') for x in content]
                    writer.writerow((lines[0], class_name))

    fp.close()

    train_gcs_url = upload_blob(
        bucket_name, train_data_fn, os.path.join(destination_blob_prefix, "train.csv")
    )

    return train_gcs_url

### Download data

In [13]:
data_dir = download_data(DATA_DIR)
print(f"Data is downloaded to: {data_dir}")

6053168/6053168 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Data is downloaded to: ./mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag/data


In [14]:
!ls $data_dir

README.md  stack_overflow_16k.tar.gz  test  train


In [15]:
!ls $data_dir/train

csharp	java  javascript  python


### Prepare csv files for creating managed dataset

#### Create csv files using data content

In [16]:
gcs_source_train_url = upload_train_data_to_gcs(
    train_data_dir=os.path.join(data_dir, "train"),
    bucket_name=BUCKET_NAME,
    destination_blob_prefix=f"{GCS_PREFIX}/data",
)

print(f"Train data content is loaded to {gcs_source_train_url}")

Train data content is loaded to gs://k-bucket-vertexai-service-project-sdk-tf-managed/mbsdk_custom-py-pkg-training/keras-text-class-stack-overflow-tag/data/train.csv


In [17]:
!gsutil ls gs://$BUCKET_NAME/$GCS_PREFIX/data

gs://k-bucket-vertexai-service-project-sdk-tf-managed/mbsdk_custom-py-pkg-training/keras-text-class-stack-overflow-tag/data/train.csv


# Create Custom Training Python Package

Before you can perform custom training with a pre-built container, you must create a [Python Source Distribution](https://docs.python.org/3/distutils/sourcedist.html) that contains your training application and upload it to a Cloud Storage bucket that your Google Cloud project can access.

You create a directory and write all of our package build artifacts into that folder.

In [18]:
PYTHON_PACKAGE_APPLICATION_DIR = f"{TASK_NAME}/trainer"

!mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR
!touch $PYTHON_PACKAGE_APPLICATION_DIR/__init__.py

### Write the Training Script

In [110]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/task.py


import os
import argparse

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import json
import tqdm

VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

def str2bool(v):
  if isinstance(v, bool):
    return v
  if v.lower() in ('yes', 'true', 't', 'y', '1'):
    return True
  elif v.lower() in ('no', 'false', 'f', 'n', '0'):
    return False
  else:
    raise argparse.ArgumentTypeError('Boolean value expected.')

def build_model(num_classes, loss, optimizer, metrics, vectorize_layer):
  # vocab_size is VOCAB_SIZE + 1 since 0 is used additionally for padding.
  model = tf.keras.Sequential([
      vectorize_layer,
      layers.Embedding(VOCAB_SIZE + 1, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_classes),
      layers.Activation('softmax')
  ])
  model.compile(
      loss=loss,
      optimizer=optimizer,
      metrics=metrics)

  return model

def get_string_labels(predicted_scores_batch, class_names):
  predicted_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(class_names, predicted_labels)
  return predicted_labels

def predict(export_model, class_names, inputs):
  predicted_scores = export_model.predict(inputs)
  predicted_labels = get_string_labels(predicted_scores, class_names)
  return predicted_labels

def parse_args():
  parser = argparse.ArgumentParser(
      description='Keras Text Classification on Stack Overflow Questions')
  parser.add_argument(
      '--epochs', default=25, type=int, help='number of training epochs')
  parser.add_argument(
      '--batch-size', default=16, type=int, help='mini-batch size')
  parser.add_argument(
      '--model-dir', default=os.getenv('AIP_MODEL_DIR'), type=str, help='model directory')
  parser.add_argument(
      '--data-dir', default='./data', type=str, help='data directory')
  parser.add_argument(
      '--test-run', default=False, type=str2bool, help='test run the training application, i.e. 1 epoch for training using sample dataset')
  parser.add_argument(
      '--model-version', default=1, type=int, help='model version')
  args = parser.parse_args()
  return args

def load_aip_dataset(aip_data_uri_pattern, batch_size, class_names, test_run, shuffle=True, seed=42):

  data_file_urls = list()
  labels = list()

  class_indices = dict(zip(class_names, range(len(class_names))))
  num_classes = len(class_names)

  for aip_data_uri in tqdm.tqdm(tf.io.gfile.glob(pattern=aip_data_uri_pattern)):
    with tf.io.gfile.GFile(name=aip_data_uri, mode='r') as gfile:
      for line in gfile.readlines():
        line = json.loads(line)
        data_file_urls.append(line['textContent'])
        classification_annotation = line['classificationAnnotations'][0]
        label = classification_annotation['displayName']
        labels.append(class_indices[label])
        if test_run:
          break

  data = list()
  for data_file_url in tqdm.tqdm(data_file_urls):
    with tf.io.gfile.GFile(name=data_file_url, mode='r') as gf:
      txt = gf.read()
      data.append(txt)

  print(f' data files count: {len(data_file_urls)}')
  print(f' data count: {len(data)}')
  print(f' labels count: {len(labels)}')

  dataset = tf.data.Dataset.from_tensor_slices(data)
  label_ds = tf.data.Dataset.from_tensor_slices(labels)
  label_ds = label_ds.map(lambda x: tf.one_hot(x, num_classes))

  dataset = tf.data.Dataset.zip((dataset, label_ds))

  if shuffle:
    # Shuffle locally at each iteration
    dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names

  return dataset

def main():

  args = parse_args()

  class_names = ['csharp', 'java', 'javascript', 'python']
  class_indices = dict(zip(class_names, range(len(class_names))))
  num_classes = len(class_names)
  print(f' class names: {class_names}')
  print(f' class indices: {class_indices}')
  print(f' num classes: {num_classes}')

  epochs = 1 if args.test_run else args.epochs

  aip_model_dir = os.environ.get('AIP_MODEL_DIR')
  aip_data_format = os.environ.get('AIP_DATA_FORMAT')
  aip_training_data_uri = os.environ.get('AIP_TRAINING_DATA_URI')
  aip_validation_data_uri = os.environ.get('AIP_VALIDATION_DATA_URI')
  aip_test_data_uri = os.environ.get('AIP_TEST_DATA_URI')

  print(f"aip_model_dir: {aip_model_dir}")
  print(f"aip_data_format: {aip_data_format}")
  print(f"aip_training_data_uri: {aip_training_data_uri}")
  print(f"aip_validation_data_uri: {aip_validation_data_uri}")
  print(f"aip_test_data_uri: {aip_test_data_uri}")

  print('Loading AIP dataset')
  train_ds = load_aip_dataset(
      aip_training_data_uri, args.batch_size, class_names, args.test_run)
  print('AIP training dataset is loaded')
  val_ds = load_aip_dataset(
      aip_validation_data_uri, 1, class_names, args.test_run)
  print('AIP validation dataset is loaded')
  test_ds = load_aip_dataset(
      aip_test_data_uri, 1, class_names, args.test_run)
  print('AIP test dataset is loaded')

  vectorize_layer = TextVectorization(
      max_tokens=VOCAB_SIZE,
      output_mode='int',
      output_sequence_length=MAX_SEQUENCE_LENGTH)

  train_text = train_ds.map(lambda text, labels: text)
  vectorize_layer.adapt(train_text)
  print('The vectorize_layer is adapted')


  print('Build model')
  optimizer = 'adam'
  metrics = ['accuracy']

  model = build_model(
      num_classes, losses.CategoricalCrossentropy(from_logits=True), optimizer, metrics, vectorize_layer)

  history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
  history = history.history

  print('Training accuracy: {acc}, loss: {loss}'.format(
      acc=history['accuracy'][-1], loss=history['loss'][-1]))
  print('Validation accuracy: {acc}, loss: {loss}'.format(
      acc=history['val_accuracy'][-1], loss=history['val_loss'][-1]))

  loss, accuracy = model.evaluate(test_ds)
  print('Test accuracy: {acc}, loss: {loss}'.format(
      acc=accuracy, loss=loss))

  inputs = [
      "how do I extract keys from a dict into a list?",  # python
      "debug public static void main(string[] args) {...}",  # java
  ]
  predicted_labels = predict(model, class_names, inputs)
  for input, label in zip(inputs, predicted_labels):
    print(f'Question: {input}')
    print(f'Predicted label: {label.numpy()}')

  model_export_path = os.path.join(args.model_dir, str(args.model_version))
  model.save(model_export_path)
  print(f'Model version {args.model_version} is exported to {args.model_dir}')

  loaded = tf.saved_model.load(model_export_path)
  input_name = list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]
  print(f'Serving function input: {input_name}')

  return

if __name__ == '__main__':
  main()


Overwriting mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag/trainer/task.py


### Build Package

In [111]:
%%writefile {TASK_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup

setup(
    name='trainer',
    version='0.1',
    packages=find_packages(),
    install_requires=(),
    include_package_data=True,
    description='My training application.'
)

Overwriting ./mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag/setup.py


In [112]:
!ls $TASK_DIR

data  dist  setup.py  trainer  trainer.egg-info


In [113]:
!cd $TASK_DIR && python3 setup.py sdist --formats=gztar

running sdist
running egg_info
writing trainer.egg-info/PKG-INFO
writing dependency_links to trainer.egg-info/dependency_links.txt
writing top-level names to trainer.egg-info/top_level.txt
reading manifest file 'trainer.egg-info/SOURCES.txt'
writing manifest file 'trainer.egg-info/SOURCES.txt'

running check
creating trainer-0.1
creating trainer-0.1/trainer
creating trainer-0.1/trainer.egg-info
copying files to trainer-0.1...
copying setup.py -> trainer-0.1
copying trainer/__init__.py -> trainer-0.1/trainer
copying trainer/task.py -> trainer-0.1/trainer
copying trainer.egg-info/PKG-INFO -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/SOURCES.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/dependency_links.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/top_level.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/SOURCES.txt -> trainer-0.1/trainer.egg-info
Writing trainer-0.1/setup.cfg
Creating tar archive
removing 'trainer-0.1' (and everythi

In [114]:
!ls -ltr $TASK_DIR/dist/trainer-0.1.tar.gz

-rw-r--r-- 1 jupyter jupyter 3243 Jul  3 13:32 ./mbsdk_custom-py-pkg-training_keras-text-class-stack-overflow-tag/dist/trainer-0.1.tar.gz


### Upload the Package to GCS

In [115]:
destination_blob_name = f"custom-training-python-package/{APP_NAME}/trainer-0.1.tar.gz"
source_file_name = f"{TASK_DIR}/dist/trainer-0.1.tar.gz"

python_package_gcs_uri = upload_blob(
    BUCKET_NAME, source_file_name, destination_blob_name
)
python_module_name = "trainer.task"

print(f"Custom Training Python Package is uploaded to: {python_package_gcs_uri}")

Custom Training Python Package is uploaded to: gs://k-bucket-vertexai-service-project-sdk-tf-managed/custom-training-python-package/keras-text-class-stack-overflow-tag/trainer-0.1.tar.gz


# Create TensorFlow Serving container

In [116]:
TF_SERVING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/tf-serving"

Download the TensorFlow Serving Docker image.

In [117]:
IS_COLAB = "google.colab" in sys.modules
if not IS_COLAB:
    !docker pull tensorflow/serving:latest
else:
    # install docker daemon
    ! apt-get -qq install docker.io

latest: Pulling from tensorflow/serving

386c2fff: Pulling fs layer 
ae012922: Pulling fs layer 
c9828151: Pulling fs layer 
63dea99c: Pulling fs layer 
Digest: sha256:121b8f8e1bc56ccfdac97efd4ce513f85b28b36a4619c1cd7abb538ecaf42cd8[3A
Status: Downloaded newer image for tensorflow/serving:latest
docker.io/tensorflow/serving:latest


Configure docker authentication with Container Registry


In [118]:
! gcloud auth configure-docker gcr.io --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: gcr.io
gcloud credential helpers already registered correctly.


Create a tag for registering the image and register the image with Cloud Container Registry (gcr.io).

In [119]:
if not IS_COLAB:
    !docker tag tensorflow/serving:latest $TF_SERVING_CONTAINER_IMAGE_URI
    !docker push $TF_SERVING_CONTAINER_IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/vertexai-service-project/tf-serving]

72ab6ed8: Preparing 
8de24927: Preparing 
ffd079dd: Preparing 
aa3439d0: Preparing 
ffd079dd: Pushed   650.4MB/650.4MBlatest: digest: sha256:cd8b23bd34dc6a68454a5cdb2da8f13507e54240537488dbce35c5fceaf50d41 size: 1367


In [120]:
%%bash -s $IS_COLAB $TF_SERVING_CONTAINER_IMAGE_URI
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull tensorflow/serving:latest
docker tag tensorflow/serving:latest $2
docker push $2
kill $(jobs -p)

# Run custom Python package training with managed text dataset

In [121]:
dataset_display_name = f"temp-{APP_NAME}-content"
gcs_source = gcs_source_train_url

####  Create a dataset with csv file

In [33]:
dataset = aiplatform.TextDataset.create(
    display_name=dataset_display_name,
    gcs_source=gcs_source,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

Creating TextDataset
Create TextDataset backing LRO: projects/314481442207/locations/us-central1/datasets/7661545456412917760/operations/8492606681702727680
TextDataset created. Resource name: projects/314481442207/locations/us-central1/datasets/7661545456412917760
To use this TextDataset in another session:
ds = aiplatform.TextDataset('projects/314481442207/locations/us-central1/datasets/7661545456412917760')
Importing TextDataset data: projects/314481442207/locations/us-central1/datasets/7661545456412917760
Import TextDataset data backing LRO: projects/314481442207/locations/us-central1/datasets/7661545456412917760/operations/7104759124658749440
TextDataset data imported. Resource name: projects/314481442207/locations/us-central1/datasets/7661545456412917760


## Launch a training job and create a model on Vertex AI

Next, you train a model with the Python package you just built.

### Config a Training Job

In [2]:
MODEL_NAME = APP_NAME
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-11:latest"
)

NameError: name 'APP_NAME' is not defined

You need to specify the Python package that was built and uploaded to GCS, the module name of the Python package, the pre-built training container image uri for training, and in this example,  are using TensorFlow serving container for prediction.

In [123]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"temp_{TASK_NAME}_tf-serving",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
    model_serving_container_image_uri=TF_SERVING_CONTAINER_IMAGE_URI,
    model_serving_container_command=["/usr/bin/tensorflow_model_server"],
    model_serving_container_args=[
        f"--model_name={MODEL_NAME}",
        "--model_base_path=$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    model_serving_container_predict_route=f"/v1/models/{MODEL_NAME}:predict",
    model_serving_container_health_route=f"/v1/models/{MODEL_NAME}",
)

### Run the Training Job

In [124]:
model = job.run(
    dataset=dataset,
    annotation_schema_uri=aiplatform.schema.dataset.annotation.text.classification,
    args=["--epochs", "1"],##TODO, change 1 to 50
    replica_count=1,
    model_display_name=f"temp_{TASK_NAME}_tf-serving",
    sync=False,
)

In [125]:
model.wait()

# Deploy a Model and Create an Endpoint on Vertex AI

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [126]:
endpoint = model.deploy(machine_type="n1-standard-4", sync=False)

In [127]:
endpoint.wait()

## Predict on the Endpoint

In [128]:
class_names = ["csharp", "java", "javascript", "python"]

class_ids = range(len(class_names))

class_indices = dict(zip(class_names, class_ids))
class_maps = dict(zip(class_ids, class_names))
print(f"Class Indices: {class_indices}")
print(f"Class Maps:    {class_maps}")

Class Indices: {'csharp': 0, 'java': 1, 'javascript': 2, 'python': 3}
Class Maps:    {0: 'csharp', 1: 'java', 2: 'javascript', 3: 'python'}


In [129]:
text_inputs = [
    "how do I extract keys from a dict into a list?",  # python
    "debug public static void main(string[] args) {...}",  # java
]

In [130]:
import numpy as np

predictions = endpoint.predict(instances=text_inputs)
for text, predicted_scores in zip(text_inputs, predictions.predictions):
    class_id = np.argmax(predicted_scores)
    class_name = class_maps[class_id]
    print(f"Question: {text}")
    print(f"Predicted Tag: {class_name}\n")

AttributeError: 'google.protobuf.pyext._message.RepeatedCompositeCo' object has no attribute 'WhichOneof'

# Batch Prediction Job on the Model

In [ ]:
import json

import tensorflow as tf


def upload_test_data_to_gcs(test_data_dir, test_gcs_url):
    """Create JSON file using test data content."""

    input_name = "text_vectorization_input"

    with tf.io.gfile.GFile(test_gcs_url, "w") as gf:

        for root, _, files in os.walk(test_data_dir):
            for file in files:
                if file.endswith(".txt"):
                    file_fn = os.path.join(root, file)
                    with open(file_fn) as f:
                        content = f.readlines()
                        lines = [x.strip().strip('"') for x in content]

                        data = {input_name: lines[0]}
                        gf.write(json.dumps(data))
                        gf.write("\n")
    return

In [ ]:
gcs_source_test_url = f"gs://{BUCKET_NAME}/{GCS_PREFIX}/data/test.json"
upload_test_data_to_gcs(
    test_data_dir=os.path.join(data_dir, "test"), test_gcs_url=gcs_source_test_url
)

print(f"Test data content is loaded to {gcs_source_test_url}")

In [ ]:
!gsutil ls $gcs_source_test_url

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"temp_{TASK_NAME}_tf-serving",
    gcs_source=gcs_source_test_url,
    gcs_destination_prefix=f"gs://{BUCKET_NAME}/{GCS_PREFIX}/batch_prediction",
    machine_type="n1-standard-4",
    sync=False,
)

In [ ]:
batch_predict_job.wait()
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_errors_stats = list()
prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.errors_stats"):
        prediction_errors_stats.append(blob.name)
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_results.append(blob.name)

In [ ]:
tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            text = line["instance"]["text_vectorization_input"][0]
            prediction = line["prediction"]
            class_id = np.argmax(prediction)
            class_name = class_maps[class_id]
            tags.append([text, class_name])

In [ ]:
import pandas as pd

tags_df = pd.DataFrame(tags, columns=["question", "tag"])
tags_df.head()

In [ ]:
tags_df["tag"].value_counts()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


In [1]:
delete_bucket = True##ToDo, make it False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Undeploy model from the endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

NameError: name 'dataset' is not defined